In [ ]:
databricks = True

In [ ]:
if databricks == False:
    import findspark
    findspark.init()

In [ ]:
import os.path

if databricks == True:
    root = "/FileStore/tables"
else:
    root = "../Data"

Créer SparkSession et SparkContext

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

* Internally, each RDD is characterized by five main properties:
 *  A list of partitions
 *  A function for computing each split
 *  A list of dependencies on other RDDs
 *  Optionally, a Partitioner for key-value RDDs (e.g. to say that the RDD is hash-partitioned)
 *  Optionally, a list of preferred locations to compute each split on (e.g. block locations for
 *    an HDFS file)
 '''

* Créer un RDD avec valeurs 'Ana' et 'Bob'
* Visualiser-le
* Recréer plusieurs fois en changeant le nombre des partiotions
* Afficher le nombre de partitions

In [ ]:
namelist = ['Ana','Bob']

In [ ]:
nameRDD = sc.parallelize(namelist)
print(nameRDD.collect())
print(nameRDD.getNumPartitions())

['Ana', 'Bob']
8


In [ ]:
nameRDD = sc.parallelize(namelist, 5)
print(nameRDD.collect())
print(nameRDD.getNumPartitions())

['Ana', 'Bob']
5


* Créer un RDD à partir du RDD avec valeurs, 'Ana', 'Bob'
* Les valeurs du nouveau RDD sont 'Ana2nd', 'Bob2nd'

In [ ]:
anotherRDD = nameRDD.map(lambda y: y + '2nd')
anotherRDD.collect()

Out[8]: ['Ana2nd', 'Bob2nd']

Créer RDD à partir du fichier 'sample.txt' et affichier les premières 5 lignes

In [ ]:
path = os.path.join(root, "sample.txt")
fileRDD = sc.textFile(path)
fileRDD.take(5)

Out[9]: ['The names "John Doe" for males, "Jane Doe" or "Jane Roe" for females, or "Jonnie Doe" and',
 '"Janie Doe" for children, or just "Doe" non-gender-specifically are used as placeholder names',
 'for a party whose true identity is unknown or must be withheld in a legal action, case, or discussion.',
 'The names are also used to refer to acorpse or hospital patient whose identity is unknown.',
 'This practice is widely used in the United States and Canada, but is rarely used in other']

Créer a un RDD de tuples à partir du RDD avec les noms  
Il devrait contenir ('Ana', 20), ('Bob', 20)

In [ ]:
pairRDD = nameRDD.map(lambda x: (x,20))
pairRDD.collect()

Out[10]: [('Ana', 20), ('Bob', 20)]

map V.S. flatmap
à partir d'un RDD avec valeurs `[['Ana','Bob'],['Caren']]`  
use map or flatMap to return:  
1: `['Ana', 'Bob', 'plus', 'Caren', 'plus']`  
2: `[['Ana', 'Bob', 'plus'], ['Caren', 'plus']]`

In [ ]:
namelist = (['Ana','Bob'],['Caren'])
nameRDD = sc.parallelize(namelist)
print(nameRDD.flatMap(lambda x: x + ['plus']).collect())
nameRDD.map(lambda x: x + ['plus']).collect()

['Ana', 'Bob', 'plus', 'Caren', 'plus']
Out[11]: [['Ana', 'Bob', 'plus'], ['Caren', 'plus']]

mapValues V.S. flatMapValues
à partir de RDD de paires `[('Ana', ['A']), ('Bob', ['B']), ('Ana', ['A2'])]` 
utiliser mapValues or flatMapValuespour obtenir
* `[('Ana', ['A', 'plus']), ('Bob', ['B', 'plus']), ('Ana', ['A2', 'plus'])]` 
* `[('Ana', 'A'),('Ana', 'plus'),('Bob', 'B'),('Bob', 'plus'),('Ana', 'A2'),('Ana', 'plus')]`

In [ ]:
pairlist = [('Ana',['A']),('Bob',['B']),('Ana',['A2'])]
pairRDD = sc.parallelize(pairlist)
pairRDD.collect()

Out[12]: [('Ana', ['A']), ('Bob', ['B']), ('Ana', ['A2'])]

In [ ]:
pairRDD.mapValues(lambda x: x + ['plus']).collect()

Out[13]: [('Ana', ['A', 'plus']), ('Bob', ['B', 'plus']), ('Ana', ['A2', 'plus'])]

In [ ]:
pairRDD.flatMapValues(lambda x: x + ['plus']).collect()

Out[14]: [('Ana', 'A'),
 ('Ana', 'plus'),
 ('Bob', 'B'),
 ('Bob', 'plus'),
 ('Ana', 'A2'),
 ('Ana', 'plus')]

Agrégation
à partir d'une liste de nombres, partitionnez-la en 3 partitions,
* Utiliser map et reduce et calculer la moyenne
* aggregate to calculate it's average

In [ ]:
numRDD = sc.parallelize([1,2,3,4,5],3)


numRDD.map(lambda x: (x, 1)).collect()


Out[15]: [(1, 1), (2, 1), (3, 1), (4, 1), (5, 1)]

In [ ]:
numRDD = sc.parallelize([1,2,3,4,5],3)


numRDD.map(lambda x: (x, 1)).collect()


acc, cnt = numRDD.map(lambda x: (x, 1)) \
    .reduce(lambda x, y: (x[0] + y[0], x[1] + y[1]))

print(acc, cnt)

#acc/cnt

15 5


### Matériaux supplémentaire

Créer un RDD [('Ana',25),('Bob',30),('Deric',40),('Ana',40)]  

Calculer la somme des valeurs pour chaque clé en utlisant
* groupbykey
* reducebykey
* foldbykey
* combinebykey

In [ ]:
pairRDD = sc.parallelize([('Ana',25),('Bob',30),('Deric',40),('Ana',40)])
# output: [('Deric', 40), ('Ana', 65), ('Bob', 30)]
pairRDD.groupByKey().map(lambda x: (x[0],sum(x[1]))).collect()
pairRDD.reduceByKey(lambda x,y: x + y).collect()
pairRDD.foldByKey(0,lambda x,y: x + y).collect()
pairRDD.combineByKey(lambda x: x, lambda x,y: x+y, lambda x,y: x+y).collect()

Out[17]: [('Ana', 65), ('Deric', 40), ('Bob', 30)]

Calculer la moyenne du RDD

In [ ]:
# output:  [('Deric', 40.0), ('Ana', 32.5), ('Bob', 30.0)]
# solution:

sumrdd = pairRDD.combineByKey(
    lambda x: (x,1),
    lambda x,y: (x[0]+y, x[1] + 1),
    lambda x,y: (x[0]+y[0],x[1]+y[1])
)
sumrdd.map(lambda x: (x[0],x[1][0]/x[1][1])).collect()

Out[18]: [('Ana', 32.5), ('Deric', 40.0), ('Bob', 30.0)]

Sauvegarder le RDD

In [ ]:
%fs rm -r /FileStore/tables/pairrdd

res0: Boolean = true

In [ ]:
path = os.path.join(root, 'pairrdd')
pairRDD.saveAsTextFile(path)